In [1]:
meas = "RNA"
layer = "raw"
census = "2024-07-01"
tissue_general = "tongue"
var_filter = None
dask_chunk_size = 0  # Non-Dask mode, by default

In [2]:
# Parameters
tissue_general = "stomach"
dask_chunk_size = 50000


In [3]:
from typing import Literal
import anndata as ad
import scanpy as sc
from somacore import AxisQuery
from tiledbsoma import Experiment, SOMATileDBContext

DEFAULT_CONFIG = {
    "vfs.s3.no_sign_request": "true",
    "vfs.s3.region": "us-west-2"
}
CENSUS_S3 = "s3://cellxgene-census-public-us-west-2/cell-census"

species = "homo_sapiens"
soma_uri = f"{CENSUS_S3}/{census}/soma"
exp_uri = f"{soma_uri}/census_data/{species}"

exp = Experiment.open(
    exp_uri,
    context=SOMATileDBContext(tiledb_config=DEFAULT_CONFIG)
)
exp

<Experiment 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens' (open for 'r') (2 items)
    'ms': 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens/ms' (unopened)
    'obs': 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens/obs' (unopened)>

In [4]:
%%time
obs_filter = f'tissue_general == "{tissue_general}"' if tissue_general else None
query = exp.axis_query(
    measurement_name=meas,
    obs_query=AxisQuery(value_filter=obs_filter) if obs_filter else None,
    var_query=AxisQuery(value_filter=var_filter) if var_filter else None,
)

CPU times: user 8.98 ms, sys: 12.6 ms, total: 21.6 ms
Wall time: 111 ms


In [5]:
%%time
query.n_obs

CPU times: user 571 ms, sys: 824 ms, total: 1.39 s
Wall time: 1.13 s


446348

## HVG

In [6]:
%%time
add = query.to_anndata(layer, dask_chunk_size=dask_chunk_size if dask_chunk_size else None)
add

CPU times: user 1.36 s, sys: 1.77 s, total: 3.13 s
Wall time: 2.94 s


AnnData object with n_obs × n_vars = 446348 × 60530
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'observation_joinid', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'

In [7]:
%%time
sc.pp.normalize_total(add)

CPU times: user 71.5 ms, sys: 14.3 ms, total: 85.8 ms
Wall time: 85.4 ms


In [8]:
%%time
sc.pp.log1p(add)

CPU times: user 2.23 ms, sys: 164 μs, total: 2.4 ms
Wall time: 2.38 ms


In [9]:
%%time
hvg = sc.pp.highly_variable_genes(add, inplace=False, subset=True)
hvg

CPU times: user 5min 57s, sys: 1min 1s, total: 6min 58s
Wall time: 4min 47s


,means,dispersions,mean_bin,dispersions_norm,highly_variable
6,0.100212,1.625945,"(-0.00467, 0.234]",0.589460,True
15,0.024124,1.667075,"(-0.00467, 0.234]",0.637106,True
32,0.017364,2.113827,"(-0.00467, 0.234]",1.154626,True
43,0.026201,1.647127,"(-0.00467, 0.234]",0.613997,True
56,0.046380,1.721354,"(-0.00467, 0.234]",0.699982,True
...,...,...,...,...,...
53011,0.023929,2.356097,"(-0.00467, 0.234]",1.435273,True
53870,0.034305,2.465700,"(-0.00467, 0.234]",1.562239,True
53899,0.026581,2.059644,"(-0.00467, 0.234]",1.091860,True
57029,0.028378,1.884718,"(-0.00467, 0.234]",0.889224,True


---
Census `tissue_general` counts, for reference:
```python
exp_obs = exp.obs.read().concat().to_pandas()
exp_obs.tissue_general.value_counts()
```
```
tissue_general
brain                       26281059
blood                       10835244
lung                         6231233
breast                       5555979
eye                          4190842
heart                        3629952
kidney                       2083054
liver                        1815408
small intestine              1237182
skin of body                 1045024
endocrine gland               979667
respiratory system            944355
bone marrow                   813373
spleen                        751041
lymph node                    717899
colon                         714413
placenta                      638786
reproductive system           570488
adrenal gland                 560114
nose                          470445
adipose tissue                468603
stomach                       446348
prostate gland                348664
fallopian tube                250178
pancreas                      250161
esophagus                     215951
digestive system              211912
musculature                   189452
large intestine               180996
pleural fluid                 179134
yolk sac                      169725
embryo                        165937
uterus                        148198
mucosa                        131978
spinal cord                   117463
exocrine gland                115722
intestine                     104490
immune system                  89046
bladder organ                  82797
vasculature                    63667
ovary                          53751
lamina propria                 45230
tongue                         45060
central nervous system         31780
esophagogastric junction       29105
axilla                         19792
pleura                         19695
skeletal system                14680
saliva                         14502
omentum                        14003
testis                         13211
tendon of semitendinosus       10533
gallbladder                     9769
scalp                           3029
ureter                          2390
Name: count, dtype: int64
```